In [15]:
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Flatten, Dense, Dropout
from tensorflow.keras.layers import Rescaling, RandomFlip, RandomRotation, RandomZoom
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.optimizers import RMSprop
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

In [16]:
new_base_dir = Path("cats_vs_dogs_small")

In [17]:
train_dataset = image_dataset_from_directory(new_base_dir / "train", image_size=(180, 180), batch_size=32)
validation_dataset = image_dataset_from_directory(new_base_dir / "validation", image_size=(180, 180), batch_size=32)
test_dataset = image_dataset_from_directory(new_base_dir / "test", image_size=(180, 180), batch_size=32)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


### Fine-tuning

In [18]:
vgg16 = VGG16(weights="imagenet", include_top=False)

**Freezing all layers until the fourth from the last**

In [19]:
vgg16.trainable = True

for layer in vgg16.layers[:-4]:
    layer.trainable = False

**Fine-tuning the model**

In [20]:
augmentation = Sequential([RandomFlip("horizontal"), RandomRotation(0.1),RandomZoom(0.2)])

In [21]:
inputs = Input(shape=(180, 180, 3))

x = augmentation(inputs)

x = preprocess_input(x)

x = vgg16(x)
x = Flatten()(x)
x = Dense(256)(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation="sigmoid")(x)

model = Model(inputs, outputs)
model.compile(loss="binary_crossentropy", optimizer=RMSprop(learning_rate=1e-5), metrics=["accuracy"])

In [22]:
callbacks = [ModelCheckpoint(filepath="fine_tuning.keras", save_best_only=True, monitor="val_loss")]
history = model.fit(train_dataset, epochs=30, validation_data=validation_dataset, callbacks=callbacks)

Epoch 1/30
63/63 [==============================] - 25s 279ms/step - loss: 4.4741 - accuracy: 0.6980 - val_loss: 0.9290 - val_accuracy: 0.8950
Epoch 2/30
63/63 [==============================] - 15s 232ms/step - loss: 1.1769 - accuracy: 0.8380 - val_loss: 0.4402 - val_accuracy: 0.9300
Epoch 3/30
63/63 [==============================] - 15s 231ms/step - loss: 0.4967 - accuracy: 0.8900 - val_loss: 0.3091 - val_accuracy: 0.9440
Epoch 4/30
63/63 [==============================] - 15s 231ms/step - loss: 0.3569 - accuracy: 0.9165 - val_loss: 0.2757 - val_accuracy: 0.9520
Epoch 5/30
63/63 [==============================] - 15s 233ms/step - loss: 0.2956 - accuracy: 0.9230 - val_loss: 0.2278 - val_accuracy: 0.9590
Epoch 6/30
63/63 [==============================] - 15s 233ms/step - loss: 0.2438 - accuracy: 0.9300 - val_loss: 0.2076 - val_accuracy: 0.9630
Epoch 7/30
63/63 [==============================] - 15s 235ms/step - loss: 0.1981 - accuracy: 0.9490 - val_loss: 0.2078 - val_accuracy: 0.9640

In [24]:
model = load_model("fine_tuning.keras")
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

63/63 [==============================] - 5s 72ms/step - loss: 0.1795 - accuracy: 0.9625
Test accuracy: 0.962
